## Import library

In [1]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)

import data_load
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import utils 
from tempfile import TemporaryFile

# save numpy array as npy file
from numpy import asarray
from numpy import save

from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from geneticalgorithm import geneticalgorithm as ga

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[MLENS] backend: threading


In [4]:
tf.__version__

'2.2.0'

In [5]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"]

## Optimization with Adam and original function

In [6]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.001])
    delta = tf.Variable(100., trainable = True)
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(80000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [5]:

PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
losses = []
for index_ in tqdm(range(1, 101)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    losses.append(loss.numpy())
    print("Data index {}. Delta {:.2f}. Loss {:.2f}".format(index_, delta_r, loss.numpy()))

  0%|          | 0/100 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
np.mean(losses)

In [ ]:
data = asarray(delta_seq)
save('delta.npy', delta_seq)

## Optimization with list of delta

In [14]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.001])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0.0, 
        stddev=1.0, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(50000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [18]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
for index_ in tqdm(range(1, 10)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    print(delta_r.shape)
    delta_seq.append(delta_r)
    print("Data index {}. Loss {:.2f}".format(index_, loss.numpy()))
 

 11%|█         | 1/9 [04:49<38:39, 289.88s/it]

(672,)
Data index 1. Loss 0.00


 22%|██▏       | 2/9 [04:54<23:50, 204.32s/it]

The performance reach MAE: 0.001. Cancelling the training at step 31
(672,)
Data index 2. Loss 0.00


 33%|███▎      | 3/9 [09:35<22:43, 227.29s/it]

...

(672,)
Data index 7. Loss 0.05


 89%|████████▉ | 8/9 [33:31<04:37, 277.51s/it]

(672,)
Data index 8. Loss 0.02


100%|██████████| 9/9 [38:27<00:00, 256.36s/it]

(672,)
Data index 9. Loss 0.26


In [21]:
np.array(delta_seq).shape

In [22]:
save('list_delta_.npy', delta_seq)






## Optimization for individual treatment effect

In [8]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence

def optimization(data):
    threhold = tf.constant([0.01])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0, 
        stddev=10, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = data['mu1'] - data['mu0']
    
    for i in range(100000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.keras.losses.MAE(true_effect, influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            if i % 1000 == 0:
                print("Epoch {}. Loss {:.5f}".format(i, loss.numpy()))
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

In [9]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
for index_ in tqdm(range(13, 14)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    print("Data index {}. Loss {:.2f}".format(index_, loss.numpy()))

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0. Loss 128.21106
Epoch 1000. Loss 35.25104
Epoch 2000. Loss 26.33487
Epoch 3000. Loss 22.03906
Epoch 4000. Loss 19.51452
Epoch 5000. Loss 17.79730
Epoch 6000. Loss 16.47934
Epoch 7000. Loss 15.40867
Epoch 8000. Loss 14.58724
Epoch 9000. Loss 13.90437
Epoch 10000. Loss 13.29880
Epoch 11000. Loss 12.75287
Epoch 12000. Loss 12.24901
Epoch 13000. Loss 11.83835
Epoch 14000. Loss 11.50295
Epoch 15000. Loss 11.18480
Epoch 16000. Loss 10.90050
Epoch 17000. Loss 10.66358
Epoch 18000. Loss 10.45095
Epoch 19000. Loss 10.25655
Epoch 20000. Loss 10.07695
Epoch 21000. Loss 9.91892
Epoch 22000. Loss 9.77304
Epoch 23000. Loss 9.63611
Epoch 24000. Loss 9.50814
Epoch 25000. Loss 9.38981
Epoch 26000. Loss 9.27811
Epoch 27000. Loss 9.17255
Epoch 28000. Loss 9.07324
Epoch 29000. Loss 8.97934
Epoch 30000. Loss 8.89013
Epoch 31000. Loss 8.80546
Epoch 32000. Loss 8.72501
Epoch 33000. Loss 8.64893
Epoch 34000. Loss 8.57677
Epoch 35000. Loss 8.50806
Epoch 36000. Loss 8.44242
Epoch 37000. Loss 8.37966
Epo

100%|██████████| 1/1 [09:38<00:00, 578.35s/it]

Data index 13. Loss 6.87


In [12]:
save('individual_list_delta_.npy', delta_seq)

## Optimization for individual treatment effects using genetic algorithm

In [6]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence


def fitness_function(delta, solution_idx):
    influence = incre_ps(delta, data)
    mae = -np.sqrt(np.mean((true_effect - influence)**2))
    return np.mean(mae)



In [7]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
data = utils.load_data(PATH_TRAIN, 1)

## Fit treatment
model_t = LogisticRegression()
model_t.fit(data[cov], data[treatment])

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data[features], data[outcome])

data['p1'] = model_t.predict_proba(data[cov])[:,1]
data['p0'] = 1 - data['p1']

## Compute counterfactual outcome with no treatment
data_pos = data.copy()
data_pos[treatment] = 1
data['cf1'] = model_y.predict(data_pos[features])

## Compute counterfactual outcome with treatment
data_neg = data.copy()
data_neg[treatment] = 0
data['cf0'] = model_y.predict(data_neg[features])

data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                      (1 - data['p1']))

data['w0'] = data['ips_weight']*data[treatment]
data['w1'] = data['ips_weight']*(1 - data[treatment])


In [8]:
true_effect = data['mu1'] - data['mu0']

In [ ]:
import pygad
import numpy

"""
Given the following function:
    y = f(w1:w6) = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + 6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7) and y=44
What are the best values for the 6 weights (w1 to w6)? We are going to use the genetic algorithm to optimize this function.
"""

#function_inputs = [4,-2,3.5,5,-11,-4.7] # Function inputs.
#desired_output = 44 # Function output.

function_inputs = np.random.rand(data.shape[0]) # Function inputs.

num_generations = 5000 # Number of generations.
num_parents_mating = 100 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 2500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -100
init_range_high = 100

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "single_point" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 10 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]

# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()


Generation = 1
Fitness    = -4.678022693269469
Change     = -4.678022693269469


Generation = 2
Fitness    = -4.199850401935463
Change     = 0.47817229133400563


Generation = 3
Fitness    = -4.175215930641381
Change     = 0.024634471294081806


Generation = 4
Fitness    = -4.164173382619324
Change     = 0.01104254802205773


Generation = 5
Fitness    = -4.145008518945877
Change     = 0.019164863673446497


Generation = 6
Fitness    = -4.125295674564733
Change     = 0.019712844381143846


Generation = 7
Fitness    = -4.091384158461873
Change     = 0.033911516102860695


Generation = 8
Fitness    = -4.0773945236537825
Change     = 0.013989634808090123


Generation = 9
Fitness    = -4.060551485801685
Change     = 0.016843037852097176


Generation = 10
Fitness    = -4.045882846912978
Change     = 0.014668638888707619


Generation = 11
Fitness    = -4.017605397804122
Change     = 0.02827744910885599


Generation = 12
Fitness    = -4.002593999633785
Change     = 0.015011398170337031


Gener

Fitness    = -3.2487751076975893
Change     = 0.002152706978776564


Generation = 99
Fitness    = -3.2476717122637737
Change     = 0.001103395433815546


Generation = 100
Fitness    = -3.2433570910386464
Change     = 0.004314621225127269


Generation = 101
Fitness    = -3.2433570910386464
Change     = 0.0


Generation = 102
Fitness    = -3.2417941621188766
Change     = 0.0015629289197698348


Generation = 103
Fitness    = -3.2404972114672135
Change     = 0.0012969506516631313


Generation = 104
Fitness    = -3.2330676212219482
Change     = 0.0074295902452652385


Generation = 105
Fitness    = -3.2317355739179376
Change     = 0.0013320473040105973


Generation = 106
Fitness    = -3.225536795147878
Change     = 0.0061987787700594765


Generation = 107
Fitness    = -3.220832120905167
Change     = 0.00470467424271126


Generation = 108
Fitness    = -3.2175192729674227
Change     = 0.0033128479377442233


Generation = 109
Fitness    = -3.213493802536938
Change     = 0.004025470430484734


G

Fitness    = -2.969300703602172
Change     = 0.0020770411382566856


Generation = 195
Fitness    = -2.96367913799988
Change     = 0.005621565602292211


Generation = 196
Fitness    = -2.9629282915624384
Change     = 0.0007508464374414991


Generation = 197
Fitness    = -2.961246924174396
Change     = 0.0016813673880422542


Generation = 198
Fitness    = -2.9571812784842857
Change     = 0.004065645690110475


Generation = 199
Fitness    = -2.9557680237697044
Change     = 0.0014132547145813135


Generation = 200
Fitness    = -2.955205713731449
Change     = 0.0005623100382554647


Generation = 201
Fitness    = -2.953049549043952
Change     = 0.0021561646874967977


Generation = 202
Fitness    = -2.9486446088490927
Change     = 0.004404940194859375


Generation = 203
Fitness    = -2.9437507843523094
Change     = 0.004893824496783328


Generation = 204
Fitness    = -2.9425194392880107
Change     = 0.0012313450642986723


Generation = 205
Fitness    = -2.940469475790029
Change     = 0.002049

Change     = 0.0025802463062811754


Generation = 290
Fitness    = -2.7901849081590053
Change     = 0.002140598235198077


Generation = 291
Fitness    = -2.7895823570497886
Change     = 0.000602551109216698


Generation = 292
Fitness    = -2.7880023843524797
Change     = 0.0015799726973089179


Generation = 293
Fitness    = -2.786312885001304
Change     = 0.0016894993511757939


Generation = 294
Fitness    = -2.7837804461057067
Change     = 0.0025324388955971777


Generation = 295
Fitness    = -2.7828230381354597
Change     = 0.0009574079702470151


Generation = 296
Fitness    = -2.781857783992708
Change     = 0.0009652541427516148


Generation = 297
Fitness    = -2.7790048495623694
Change     = 0.0028529344303387383


Generation = 298
Fitness    = -2.778437250565929
Change     = 0.0005675989964402106


Generation = 299
Fitness    = -2.7756590622876596
Change     = 0.002778188278269589


Generation = 300
Fitness    = -2.7749638177496987
Change     = 0.0006952445379608818


Generation =

Generation = 385
Fitness    = -2.659425452339418
Change     = 0.003546715870828887


Generation = 386
Fitness    = -2.658974206380672
Change     = 0.0004512459587462381


Generation = 387
Fitness    = -2.65833184640561
Change     = 0.0006423599750617903


Generation = 388
Fitness    = -2.657715207896073
Change     = 0.0006166385095371929


Generation = 389
Fitness    = -2.655975913700711
Change     = 0.0017392941953620067


Generation = 390
Fitness    = -2.655975913700711
Change     = 0.0


Generation = 391
Fitness    = -2.654989673627185
Change     = 0.0009862400735256571


Generation = 392
Fitness    = -2.654862398500928
Change     = 0.00012727512625732018


Generation = 393
Fitness    = -2.652128979361426
Change     = 0.002733419139501869


Generation = 394
Fitness    = -2.650705294310938
Change     = 0.001423685050487844


Generation = 395
Fitness    = -2.6490411750827834
Change     = 0.0016641192281547923


Generation = 396
Fitness    = -2.647928332787551
Change     = 0.0011128422

...

Generation = 674
Fitness    = -2.431756674703301
Change     = 0.0005819567595386488


Generation = 675
Fitness    = -2.4310683528538704
Change     = 0.0006883218494304622


Generation = 676
Fitness    = -2.430964520978126
Change     = 0.00010383187574447561


Generation = 677
Fitness    = -2.430619328496023
Change     = 0.00034519248210296283


Generation = 678
Fitness    = -2.4304559345161834
Change     = 0.0001633939798395545


Generation = 679
Fitness    = -2.429954066609869
Change     = 0.0005018679063142351


Generation = 680
Fitness    = -2.429684288025296
Change     = 0.00026977858457311044


Generation = 681
Fitness    = -2.4295053122919854
Change     = 0.00017897573331060457


Generation = 682
Fitness    = -2.4288494644755656
Change     = 0.0006558478164198078


Generation = 683
Fitness    = -2.4283095125600065
Change     = 0.0005399519155591115


Generation = 684
Fitness    = -2.427803621537584
Change     = 0.0005058910224224888


Generation = 685
Fitness    = -2.427803621537

Generation = 771
Fitness    = -2.3844253828266835
Change     = 0.0006429313899780631


Generation = 772
Fitness    = -2.3843759181782076
Change     = 4.9464648475883166e-05


Generation = 773
Fitness    = -2.3830309014570696
Change     = 0.0013450167211379593


Generation = 774
Fitness    = -2.383017509535218
Change     = 1.3391921851457766e-05


Generation = 775
Fitness    = -2.38236173155939
Change     = 0.0006557779758282756


Generation = 776
Fitness    = -2.382068139395527
Change     = 0.0002935921638629324


Generation = 777
Fitness    = -2.381110774892725
Change     = 0.0009573645028018163


Generation = 778
Fitness    = -2.3809877421074455
Change     = 0.00012303278527969752


Generation = 779
Fitness    = -2.3806976601454792
Change     = 0.0002900819619662265


Generation = 780
Fitness    = -2.380129400443089
Change     = 0.0005682597023901614


Generation = 781
Fitness    = -2.379671896480139
Change     = 0.00045750396294996776


Generation = 782
Fitness    = -2.3788801676650

Change     = 0.0


Generation = 868
Fitness    = -2.342310568410251
Change     = 0.0007299064652737108


Generation = 869
Fitness    = -2.3422609252518876
Change     = 4.9643158363643636e-05


Generation = 870
Fitness    = -2.341598445074735
Change     = 0.0006624801771524247


Generation = 871
Fitness    = -2.341598445074735
Change     = 0.0


Generation = 872
Fitness    = -2.3412293657766448
Change     = 0.0003690792980903801


Generation = 873
Fitness    = -2.34059764168391
Change     = 0.000631724092734931


Generation = 874
Fitness    = -2.3402504999772376
Change     = 0.0003471417066722715


Generation = 875
Fitness    = -2.339189492060705
Change     = 0.0010610079165327413


Generation = 876
Fitness    = -2.338457667390098
Change     = 0.0007318246706069509


Generation = 877
Fitness    = -2.338155105444994
Change     = 0.0003025619451038075


Generation = 878
Fitness    = -2.3376231592364305
Change     = 0.0005319462085635607


Generation = 879
Fitness    = -2.3370858226076083


Change     = 0.0014798058071701092


Generation = 966
Fitness    = -2.299620143867724
Change     = 0.0


Generation = 967
Fitness    = -2.2992847380521613
Change     = 0.0003354058155626838


Generation = 968
Fitness    = -2.2991698702567906
Change     = 0.0001148677953706212


Generation = 969
Fitness    = -2.299025802281284
Change     = 0.00014406797550670092


Generation = 970
Fitness    = -2.2985361758513414
Change     = 0.0004896264299425468


Generation = 971
Fitness    = -2.2982838010781843
Change     = 0.00025237477315709356


Generation = 972
Fitness    = -2.297343030915379
Change     = 0.000940770162805471


Generation = 973
Fitness    = -2.2971772366524523
Change     = 0.00016579426292651078


Generation = 974
Fitness    = -2.297091580664298
Change     = 8.565598815435393e-05


Generation = 975
Fitness    = -2.2965095193513654
Change     = 0.0005820613129325913


Generation = 976
Fitness    = -2.296440984818042
Change     = 6.853453332356096e-05


Generation = 977
Fitness   

Generation = 1062
Fitness    = -2.264959970922695
Change     = 0.0005452208505398737


Generation = 1063
Fitness    = -2.264694882927575
Change     = 0.00026508799511981707


Generation = 1064
Fitness    = -2.264483835152748
Change     = 0.0002110477748269446


Generation = 1065
Fitness    = -2.264315996240793
Change     = 0.0001678389119552648


Generation = 1066
Fitness    = -2.2641071532646313
Change     = 0.00020884297616152736


Generation = 1067
Fitness    = -2.2635681492466757
Change     = 0.000539004017955591


Generation = 1068
Fitness    = -2.2631286543104316
Change     = 0.000439494936244067


Generation = 1069
Fitness    = -2.2629006677369152
Change     = 0.00022798657351641438


Generation = 1070
Fitness    = -2.2627361361930984
Change     = 0.0001645315438167927


Generation = 1071
Fitness    = -2.2626422987090393
Change     = 9.383748405911163e-05


Generation = 1072
Fitness    = -2.261987751448078
Change     = 0.000654547260961369


Generation = 1073
Fitness    = -2.261